In [2]:
import multiprocess as mp
import tqdm
import geopandas as gpd
import pandas as pd
import shapely
import fiona
shapely.speedups.enable()

### To do
- reduce coordinate precision to speed things up


In [3]:
# Load in the isochrones for the East Midlands
df = gpd.read_file('isochrones/west_midlands.geojson')

In [5]:
#Load in the centroids from the population grid
centroids = gpd.read_file('data/gb_centroid.geojson')

In [52]:
# Calculate the total population of the UK
totalpop=centroids['population'].sum()
totalpop

60553399

In [50]:
# define a function that takes a isochrone, filters the centroids within the isochrone and then adds the population. Returns true is that isochrone contains 90% of the GB population
def inthegoldentriangle(isochrone):
    within=centroids.loc[centroids.within(isochrone)]
    if(within['population'].sum()/totalpop>0.9):
        return True
    else:
        return False    

In [53]:
# test the function if necessary
inthegoldentriangle(df['geometry'][4])

False

In [58]:
# run the function across all the isochrones in parallel, save it to a list called output
pool=mp.Pool(8) #use 8 processing threads
output = list(tqdm.tqdm_notebook(pool.imap(inthegoldentriangle2, df['geometry']), total=len(df['geometry'])))
pool.close()
pool.join()

In [60]:
# find out how many squares are within the golden triangle
sum(x==True for x in output)

703

In [61]:
# Add the column to the isochrones
df['inside']=output

In [62]:
# Here's all the isochrones that are within the golden triangle
df[df['inside']==True]

,iso_id,id,min,max,center,geometry,inside
2555,2556,1,0.0,240.0,120.0,"MULTIPOLYGON (((-2.66973 55.35001, -2.99188 55...",True
2637,2638,1,0.0,240.0,120.0,"MULTIPOLYGON (((-2.61048 55.40953, -2.93264 55...",True
2638,2639,1,0.0,240.0,120.0,"MULTIPOLYGON (((-1.62922 55.23503, -1.89677 55...",True
2639,2640,1,0.0,240.0,120.0,"MULTIPOLYGON (((-2.58088 55.38399, -2.90303 55...",True
2640,2641,1,0.0,240.0,120.0,"MULTIPOLYGON (((-1.57001 55.24352, -1.85239 55...",True
...,...,...,...,...,...,...,...
6442,6443,1,0.0,240.0,120.0,"MULTIPOLYGON (((-1.24676 55.06536, -1.56891 55...",True
6526,6527,1,0.0,240.0,120.0,"MULTIPOLYGON (((-1.37883 55.04827, -1.70099 55...",True
6532,6533,1,0.0,240.0,120.0,"MULTIPOLYGON (((-1.29082 55.05676, -1.61297 55...",True
6627,6628,1,0.0,240.0,120.0,"MULTIPOLYGON (((-1.29087 55.04787, -1.61302 55...",True


In [4]:
# attach if a centroid is within the golden triangle
lookup=gpd.read_file('lookups/west_midlands_lookup.geojson')
lookup['inside']=output


NameError: name 'output' is not defined

In [76]:
# find the centre points of the golden isochrones
goldentriangle=lookup[lookup['inside']==True]
# output those points as a geojson
goldentriangle.to_file('westmidlands-inside.geojson',driver='GeoJSON')

In [68]:
# make a geodataframe with the points
gdf = gpd.GeoDataFrame(goldentriangle, geometry=gpd.points_from_xy(goldentriangle.lon,goldentriangle.lat))
# save it to a geojson
gdf.to_file('goldenpoints-westmidlands.geojson',driver='GeoJSON')

,iso_id,id,min,max,center,population,lon,lat,geometry,inside
2555,2556,1,0.0,240.0,120.0,28,-2.186494,52.719006,POINT (-2.18649 52.71901),True
2637,2638,1,0.0,240.0,120.0,9,-2.127250,52.710095,POINT (-2.12725 52.71010),True
2638,2639,1,0.0,240.0,120.0,217,-2.112448,52.710110,POINT (-2.11245 52.71011),True
2639,2640,1,0.0,240.0,120.0,154,-2.097647,52.710123,POINT (-2.09765 52.71012),True
2640,2641,1,0.0,240.0,120.0,85,-2.053243,52.710152,POINT (-2.05324 52.71015),True
...,...,...,...,...,...,...,...,...,...,...
6442,6443,1,0.0,240.0,120.0,20,-1.729990,52.323290,POINT (-1.72999 52.32329),True
6526,6527,1,0.0,240.0,120.0,68,-1.862066,52.314527,POINT (-1.86207 52.31453),True
6532,6533,1,0.0,240.0,120.0,21,-1.774051,52.314392,POINT (-1.77405 52.31439),True
6627,6628,1,0.0,240.0,120.0,132,-1.774097,52.305402,POINT (-1.77410 52.30540),True


In [71]:
# to make a convex hull around the points, convert it to a geodataframe, and then save it
triangle=gdf.unary_union.convex_hull
hull = gpd.GeoDataFrame(geometry=gpd.GeoSeries(triangle))
hull
hull.set_crs('wgs84')
hull.to_file('goldentriangle-westmidlands.geojson',driver='GeoJSON')

To create a geojson that contains all the merged 1 square km grids that are golden.


In [11]:
goldenpoints=gpd.read_file('goldenpoints-westmidlands.geojson')

In [3]:
#read in the grid file for the WMs
wmgrid=gpd.read_file('wm_grid.geojson').to_crs('wgs84')
wmgrid=wmgrid.drop(columns=['OBJECTID','RGN20CD','RGN20NM','BNG_E','BNG_N','LONG','LAT','Shape__Area','Shape__Length'])

In [15]:
# select the square KMs, that contain golden centroids
goldengrid=gpd.sjoin(wmgrid,goldenpoints, op='contains')

In [16]:
# this is the grid as individual squares
goldengrid.to_file('wmgoldengrid.geojson',driver='GeoJSON')

In [17]:
# merge the squares together if possible
onegrid=goldengrid.dissolve(by='inside')
#and save it 
onegrid.to_file('wmgoldengridmerged.geojson',driver='GeoJSON')